
# Multi-armed Bandits

<img src="images/one-armed-bandit.jpg" width="60%">


<img src="images/k.png" width="60%">


We start by selecting action $A_1$, which is the index of the arm to use, and we
get a reward of $R_1$. We then repeat the process by selecting actions $A_2$, $A_3$, …

Let $q_*(a)$ be the real _value_ of an action $a$:
$$q_*(a) = 𝔼[R_t | A_t = a].$$


Denoting $Q_t(a)$ our estimated value of action $a$ at time $t$ (before taking
trial $t$), we would like $Q_t(a)$ to converge to $q_*(a)$. A natural way to
estimate $Q_t(a)$ is
$$Q_t(a) ≝ \frac{\textrm{sum of rewards when action }a\textrm{ is taken}}{\textrm{number of times action }a\textrm{ was taken}}.$$


Following the definition of $Q_t(a)$, we could choose a _greedy action_ $A_t$ as
$$A_t(a) ≝ {argmax}_a Q_t(a).$$


# $ε$-greedy Method

## Exploitation versus Exploration

Choosing a greedy action is _exploitation_ of current estimates. We however also
need to _explore_ the space of actions to improve our estimates.


An _$ε$-greedy_ method follows the greedy action with probability $1-ε$, and
chooses a uniformly random action with probability $ε$.

# $ε$-greedy Method
<img src="images/e_greedy.png" width="60%">

# $ε$-greedy Method

## Incremental Implementation

Let $Q_{n+1}$ be an estimate using $n$ rewards $R_1, \ldots, R_n$.

$$\begin{aligned}
Q_{n+1} &= \frac{1}{n} ∑_{i=1}^n R_i \\
    &= \frac{1}{n} (R_n + \frac{n-1}{n-1} ∑_{i=1}^{n-1} R_i) \\
    &= \frac{1}{n} (R_n + (n-1) Q_n) \\
    &= \frac{1}{n} (R_n + n Q_n - Q_n) \\
    &= Q_n + \frac{1}{n}\Big(R_n - Q_n\Big)
\end{aligned}$$


# $ε$-greedy Method Algorithm


<img src="images/bandits_algorithm.png" width="60%">
section: Non-stationary Problems
# Fixed Learning Rate

Analogously to the solution obtained for a stationary problem, we consider
$$Q_{n+1} = Q_n + α(R_n - Q_n).$$


Converges to the true action values if
$$∑_{n=1}^∞ α_n = ∞ \textrm{~~~~and~~~~}∑_{n=1}^∞ α_n^2 < ∞.$$


Biased method, because
$$Q_{n+1} = (1 - α)^n Q_1 + ∑_{i=1}^n α(1-α)^{n-i} R_i.$$


The bias can be utilized to support exploration at the start of the episode by
setting the initial values to more than the expected value of the optimal
solution.


# Optimistic Initial Values and Fixed Learning Rate


<img src="images/optimistic_values.png" width="60%">
# Upper Confidence Bound

Using same epsilon for all actions in $ε$-greedy method seems inefficient. One
possible improvement is to select action according to upper confidence bound
(instead of choosing a random action with probability $ε$):
$$A_t ≝ {argmax}_a \left[Q_t(a) + c\sqrt{\frac{\ln t}{N_t(a)}}\right].$$

The updates are then performed as before (e.g., using averaging, or fixed
learning rate $α$).


# Motivation Behind Upper Confidence Bound

Actions with little average reward are probably selected too often.


Instead of simple $ε$-greedy approach, we might try selecting an action as
little as possible, but still enough to converge.


Assuming random variables $X_i$ bounded by $[0, 1]$ and $X̄ = ∑_{i=1}^N
X_i$, (Chernoff-)Hoeffding's inequality states that
$$P(X̄ - 𝔼[X̄] ≥ δ) ≤ e^{-2nδ^2}.$$


Our goal is to choose $δ$ such that for every action,
$$P(Q_t(a) - q_*(a) ≥ δ) ≤ \left(\frac{1}{t}\right)^α.$$


We can achieve the required inequality (with $α=2$) by setting
$$δ ≥ \sqrt{(\ln t)/N_t(a)}.$$


# Asymptotical Optimality of UCB

We define _regret_ as a difference of maximum of what we could get
(i.e., repeatedly using action with maximum expectation) and
what a strategy yields, i.e.,
$$\textit{regret}_N ≝ N \max_a q_*(a) - ∑_{i=1}^N 𝔼[R_i].$$


It can be shown that regret of UCB is asymptotically optimal,
see Lai and Robbins (1985), Asymptotically Efficient Adaptive Allocation Rules.


# Upper Confidence Bound Results

<img src="images/ucb.png" width="60%">

# Gradient Bandit Algorithms

Let $H_t(a)$ be a numerical _preference_ for an action $a$ at time $t$.

We could choose actions according to softmax distribution:
$$π(A_t = a) ≝ {softmax}(a) = \frac{e^{H_t(a)}}{∑_b e^{H_t(b)}}.$$


Usually, all $H_1(a)$ are set to zero, which corresponds to random uniform
initial policy.


Using SGD and MLE loss, we can derive the following algorithm:
$$H_{t+1}(a) ← H_t(a) + αR_t([a = A_t] - π(a)).$$

# Gradient Bandit Algorithms

<img src="images/gradient_bandits.png" width="60%">

# Method Comparison

<img src="images/bandits_comparison.png" width="60%">

# Lab

In [1]:
import numpy as np

class MultiArmedBandits():
    def __init__(self, bandits, episode_length):
        self._bandits = []
        for _ in range(bandits):
            self._bandits.append(np.random.normal(0., 1.))
        self._done = True
        self._episode_length = episode_length
        #print("Initialized {}-armed bandit, maximum average reward is {}".format(bandits, np.max(self._bandits)))

    def reset(self):
        self._done = False
        self._trials = 0
        return None

    def step(self, action):
        if self._done:
            raise ValueError("Cannot step in MultiArmedBandits when there is no running episode")
        self._trials += 1
        self._done = self._trials == self._episode_length
        reward = np.random.normal(self._bandits[action], 1.)
        return None, reward, self._done, {}

In [ ]:
if __name__ == "__main__":
    # Fix random seed
    np.random.seed(42)

    # Parse arguments
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("--bandits", default=10, type=int, help="Number of bandits.")
    parser.add_argument("--episodes", default=1000, type=int, help="Training episodes.")
    parser.add_argument("--episode_length", default=1000, type=int, help="Number of trials per episode.")

    parser.add_argument("--mode", default="greedy", type=str, help="Mode to use -- greedy, ucb and gradient.")
    parser.add_argument("--alpha", default=0, type=float, help="Learning rate to use (if applicable).")
    parser.add_argument("--c", default=1., type=float, help="Confidence level in ucb.")
    parser.add_argument("--epsilon", default=0.1, type=float, help="Exploration factor (if applicable).")
    parser.add_argument("--initial", default=0, type=float, help="Initial value function levels.")
    args = parser.parse_args()

    env = MultiArmedBandits(args.bandits, args.episode_length)

    average_rewards = []
    for episode in range(args.episodes):
        env.reset()

        # TODO: Initialize required values (depending on mode).

        average_rewards.append(0)
        done = False
        while not done:
            # TODO: Action selection according to mode
            if args.mode == "greedy":
                action =
            elif args.mode == "ucb":
                action =
            elif args.mode == "gradient":
                action =

            _, reward, done, _ = env.step(action)
            average_rewards[-1] += reward / args.episode_length

            # TODO: Update parameters

    # Print out final score as mean and variance of all obtained rewards.
    print("Final score: {}, variance: {}".format(np.mean(average_rewards), np.var(average_rewards)))
